In [28]:
from IPython.display import HTML
# disables the code cell in NBviewer
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>''')

In [29]:
import sys
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
sys.path.append('../SignalBacktesting/') # god this is awful
from backtesting import backtesting
import cufflinks as cf
cf.set_config_file(offline=True, offline_show_link=False, theme='white', sharing='public', colorscale='ggplot')
from betterMT5 import connected
from datetime import datetime
import pandas as pd

In [30]:
with connected():
    test = backtesting.Backtest("../SignalBacktesting/chats/results_daniel.json", verbose=None)

In [31]:
with connected():
    results = test.run()

In [115]:
def gen_results(test_run = test.run_results, ignore = None, partials=None):
    results = pd.DataFrame(test.make_results(test_run, ignore=ignore, partials=partials))
    results.drop(results[results.result>4].index, axis=0, inplace=True)
    results.drop(results[results.sl_pips>50].index, axis=0, inplace=True)
    results.drop(results[results.sl_pips<5].index, axis=0, inplace=True)
    results.open = pd.to_datetime(results.open.apply(str), utc=True)
    results.close = pd.to_datetime(results.close.apply(str), utc=True)
    results.set_index('open', inplace=True)
    results.drop(results.loc['2021-12-22'].index, axis=0, inplace=True) #outlier
    results.reset_index(inplace=True)
    return results

In [33]:
def gen_multi(results: list, names: list, attr: str = 'result'):
    to_pass = {k: v.set_index('open') for k, v in zip(names, results)}
    return pd.concat(to_pass, axis=0).reset_index()

def get_attr_multi(multi, attr):
    return multi.pivot('open', 'level_0', attr)

## Risultati (Daniel Savage base)

In [116]:
results = gen_results()
data_loss = len(test.trades) - results.shape[0]
print(f'Data loss dopo correzione errori: {data_loss} ({data_loss/len(test.trades):.2%})')
print(f"Winrate: {results[results.result>=0]['result'].count()/len(results):.2%}")
print(f"R medio TP: {results[results.type=='TP']['result'].mean():.2f}")

Data loss dopo correzione errori: 22 (5.13%)
Winrate: 51.60%
R medio TP: 0.96


In [119]:
results.iplot(kind='scatter', mode='markers', x='close', title='Risultati (R) in base al tipo di chiusura', yTitle='R',
              y='result', categories='type', size=6, colors='red blue purple green grey black'.split(' '))

In [120]:
results_ignore_close = gen_results(ignore=['CLOSE'])
results_ignore_sl_to_be = gen_results(ignore=['SL_TO_BE'])
results_ignore_partials = gen_results(ignore=['PARTIALS'])
results_60_40 = gen_results(partials=[.6, .4])
results_40_50_10 = gen_results(partials=[.4, .5, .1])
multi = gen_multi(results=[results, results_ignore_close, results_ignore_sl_to_be, results_ignore_partials, results_60_40, results_40_50_10],
                  names='normal ignore_close ignore_be ignore_partials 60_40 40_50_10'.split(' '))

### Consideriamo le possibili combinazioni attuative: SL a BE, CLOSE e PARZIALI

In [121]:
interpolated = get_attr_multi(multi, 'result').cumsum().resample('3600T').mean().interpolate(method='spline', order=3)
interpolated.iplot(kind='scatter', title='Risultati cumulativi per variante (R)', yTitle='R',
    xTitle='date', opacity=1, zerolinecolor='black')
del interpolated

In [122]:
get_attr_multi(multi, 'result').groupby(pd.Grouper(freq='M')).sum().iplot(kind='bar', barmode='group',
    title='Risultati mensili per variante (R)', xTitle='date', yTitle='R', opacity=1)

In [123]:
get_attr_multi(multi, 'result').groupby(pd.Grouper(freq='D')).sum().iplot(kind='bar', subplots=True,
    title='Risultati giornalieri per variante (R)', xTitle='date', yTitle='R', opacity=1, bargroupgap=0.1)

In [124]:
df = get_attr_multi(multi, 'result').groupby(pd.Grouper(freq='D')).count()
df.loc[(df!=0).any(axis=1)].iplot(kind='hist', title='Distribuzione numero di trades in 1 giorno per variante', 
    xTitle='number of trades', yTitle='frequency', opacity=1, bargroupgap=0.6, linecolor='#F2F3F5')

In [125]:
df = get_attr_multi(multi, 'result').groupby(
    pd.Grouper(freq='D')).agg(['count', 'sum']).reset_index(drop=True).stack().to_dict()
final_d = {}
for n in range(1, 7):
    final_d[n] = dict()
    for k, v in df.items():
        # ex: ignore_be:
        final_d[n][k] = 0
        flag = False
        for k2, v2 in v.items():
            if k2[1] == 'count':
                if v2 == n:
                    flag = True
                    continue
                flag = False
            if flag:
                final_d[n][k] += v2

pd.DataFrame(final_d).T.iplot(kind='bar', title='Distribuzione risultati per numero di trades in 1 giorno per variante (R)',
    xTitle='number of trades', yTitle='R', opacity=1, linecolor='#F2F3F5', bargap=0.3)

In [126]:
get_attr_multi(multi, 'sl_pips').iplot(kind='hist', barmode='overlay', title='Distribuzione numero di pips di SL per variante', 
    yTitle='frequency', xTitle='pips', opacity=1, linecolor='white')

In [127]:
df = get_attr_multi(multi, 'close')
df = df.apply(lambda x: (x-df.index).dt.total_seconds()/60).reset_index().drop('open', axis=1)
df.iplot(kind='hist', barmode='group', title='Distribuzione durata dei trades in minuti per variante', yTitle='frequency', 
    xTitle='minutes', opacity=1, linecolor='#F2F3F5')

In [128]:
df = results[:]
df['timeofday'] = df['open'].dt.hour
df.groupby('timeofday').count()[['result']].iplot(kind='bar', title='Distribuzione numero di trades per ora del giorno (UTC)', 
    barmode='group', xTitle='hour of day', yTitle='frequency', opacity=1, bargroupgap=0, linecolor='#F2F3F5')

In [129]:
df.groupby('timeofday').sum()[['result']].iplot(kind='bar', title='Risultati per ora del giorno (UTC)', 
    barmode='group', xTitle='hour of day', yTitle='R', opacity=1, bargroupgap=0, linecolor='#F2F3F5')

### Proviamo a limitare il numero massimo di trades giornalieri a 2
**E usiamo la variante 60/40 considerato che si è rivelata la migliore**

In [130]:
new_results = results_60_40.set_index('open')
new_results['counter'] = 1
new_results['counter'] = new_results[['counter']].groupby(pd.Grouper(freq='D')).transform(lambda x: x.cumsum())
new_results = new_results[new_results.counter < 3]
new_results = new_results.resample('T').sum()
new_results['new_60_40'] = new_results.resample('T').sum()['result']
new_results['new_60_40_r1.5'] = new_results.resample('T').sum()['result'] * 1.5
new_results['old_normal'] = results.set_index('open').resample('T').sum()['result']
new_results = new_results[['new_60_40', 'new_60_40_r1.5', 'old_normal']].loc[(new_results.new_60_40 != 0) | (new_results.old_normal != 0) | (new_results['new_60_40_r1.5'] != 0)]
new_results.new_60_40.loc[new_results.new_60_40==0] = None
new_results['new_60_40_r1.5'].loc[new_results['new_60_40_r1.5']==0] = None

In [131]:
new_results.loc['2021-12-22']

,new_60_40,new_60_40_r1.5,old_normal
open,,,


In [132]:
df = new_results.groupby(pd.Grouper(freq='D')).count()
df.loc[(df!=0).any(axis=1)].iplot(kind='hist', title='Distribuzione numero di trades in 1 giorno', barmode='group',
    xTitle='number of trades', yTitle='frequency', opacity=1, bargroupgap=0, linecolor='#F2F3F5')

In [133]:
interpolated = new_results.cumsum().resample('3600T').mean().interpolate(method='spline', order=3)
interpolated.iplot(kind='scatter', title='Risultati cumulativi (R)', yTitle='R',
    xTitle='date', opacity=1, zerolinecolor='black')
del interpolated

In [134]:
new_results.groupby(pd.Grouper(freq='M')).sum().iplot(kind='bar', barmode='group',
    title='Risultati mensili (R)', xTitle='date', yTitle='R', opacity=1)

In [135]:
new_results.groupby(pd.Grouper(freq='D')).sum().iplot(kind='bar', subplots=True,
    title='Risultati giornalieri (R)', xTitle='date', yTitle='R', opacity=1, bargroupgap=0.1)

### Analisi Streak

In [136]:
def make_streak_df(df, by='open'):

    df = df.set_index(by).sort_index()[['result']]
        
    df['result'] = df.result.apply(lambda x: 1 if x>=0 else 0)
    grouper = (df.result != df.result.shift()).cumsum()
    df['streak_pos'] = df.groupby(grouper).cumsum()

    df['result'] = df.result.apply(lambda x: 1 if x==0 else 0) # inverts
    grouper = (df.result != df.result.shift()).cumsum()
    df['streak_neg'] = df.groupby(grouper).cumsum()[['result']]

    df['streak'] = df.streak_pos-df.streak_neg
    return df.reset_index(drop=True)[['streak']]

by_open = make_streak_df(results, by='open').rename(columns={'streak': 'by_open'})
by_close = make_streak_df(results, by='close').rename(columns={'streak': 'by_close'})
streaks = pd.concat([by_open, by_close], axis=1)
streaks.iplot(kind='scatter', title='Numero di risultati uguali consecutivi', xTitle='index', yTitle='streak count', interpolation='vh', fill=True)

In [137]:
by_close['bucket'] = abs(by_close.by_close) < abs(by_close.by_close - by_close.by_close.shift(-1))
buckets = by_close.loc[by_close.bucket][['by_close']]
buckets.value_counts().rename_axis('unique_values').reset_index(name='counts').set_index('unique_values').iplot(kind='bar', title='Distribuzione dei casi di streak consecutive', 
    yTitle='frequency', xTitle='streak max', opacity=1)